In [4]:
import h5py
import tkinter as tk
from tkinter import filedialog
import numpy as np
import tifffile
import os
SCRIPT_DIR = os.path.dirname(os.path.abspath(__vsc_ipynb_file__))
sys.path.append(os.path.dirname(SCRIPT_DIR))
from processing.processing_functions import *

root = tk.Tk()
root.withdraw()
gt_file_path = filedialog.askopenfilename()

# gt_file = h5py.File(gt_file_path, 'r+')
# gt_image = np.array(gt_file['project_data'].get('raw_image'))
# gt_label = np.array(gt_file['project_data'].get('label'))

gt_label = tifffile.imread(gt_file_path).astype(np.uint16)


root = tk.Tk()
root.withdraw()
pred_file_path = filedialog.askopenfilename()

# pred_file = h5py.File(pred_file_path, 'r+')
# pred_image = np.array(pred_file['project_data'].get('raw_image'))
# pred_label = np.array(pred_file['project_data'].get('label'))

pred_label = tifffile.imread(pred_file_path).astype(np.uint16)


In [25]:
pred_label[pred_label>2] = 2
gt_label[gt_label>2] = 0
np.unique(pred_label)

array([0, 1, 2], dtype=uint16)

In [14]:
pred_label.shape

(35, 512, 512)

In [21]:
location = np.where(gt_label == 2)

In [23]:
location[0].shape

(27924,)

In [17]:
pred_dendrite

array([0, 1, 2], dtype=uint16)

In [12]:
gt_dendrite = gt_label[location]

In [15]:
gt_dendrite

array([2, 2, 2, ..., 2, 2, 2], dtype=uint16)

In [26]:
categorical_gt = to_categorical(gt_label,3)
categorical_pred = to_categorical(pred_label,3)

In [33]:
intersection = np.count_nonzero(categorical_gt[...,2]*categorical_pred[...,2])

In [34]:
pseudo_iou = intersection/np.count_nonzero(categorical_gt[...,2])

In [35]:
pseudo_iou

0.7688010313708638

: 

In [7]:
SCRIPT_DIR = os.path.dirname(os.path.abspath(__vsc_ipynb_file__))
sys.path.append(os.path.dirname(SCRIPT_DIR))

from processing.processing_functions import *

## Calculate the Dice Score


In [17]:
from monai.metrics import compute_meandice
import torch

dim_order = (0,4,1,2,3)

pred_cat = to_categorical_torch(pred_label, 3)
gt_cat = to_categorical_torch(gt_label,3)

pred_cat = torch.permute(torch.unsqueeze(pred_cat,0),dim_order)
gt_cat = torch.permute(torch.unsqueeze(gt_cat, 0),dim_order)

In [18]:
pred_cat.shape

torch.Size([1, 3, 35, 512, 512])

In [19]:
dice_scores = compute_meandice(pred_cat, gt_cat, include_background=True)

In [22]:
dice_scores

tensor([[0.9980, 0.2619, 0.5840]])

In [23]:
pred_file.close()
gt_file.close()

: 

## Calculate Precision and Recall

## Calculate Area Under the ROC Curve